<a href="https://colab.research.google.com/github/zeze1004/AI/blob/master/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import csv
import torch
import torch.optim as optim
import csv
import torch.nn as nn

wine_path = "https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/tabular-wine/winequality-white.csv"
#wine_path = "../data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype = np.float32, delimiter = ";", skiprows = 1)
wine_data = pd.read_csv(wine_path, delimiter=';')
wineq_numpy.shape

wineq = torch.from_numpy(wineq_numpy)
wineq.shape # columns and rows
wineq.dtype

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [4]:
# quality 제거
data = wineq[:, :-1] # <1>
data

tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
        [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
        [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
        [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
        [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]])

In [7]:
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [8]:
# target을 onehot vector로 생성
target_onehot = torch.zeros(target.shape[0],10)
target_onehot.scatter_(1, target.unsqueeze(1),1.0)
target = target_onehot
target

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [13]:
n_samples = data.shape[0]
n_samples
# training data 개수
n_val = int(0.2 * n_samples)
n_val

979

In [14]:
shuffled_indices = torch.randperm(n_samples)
shuffled_indices

tensor([ 246, 4628, 1913,  ..., 3011, 2771,  392])

In [30]:
# training data, validation data 분리
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]


In [29]:
# 트레이닝 루프 정의
import torch.nn as nn

def training_loop(n_epochs, optimizer, model, loss_fn, data_train, data_val,
                  target_train, target_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(data_train) # <1>
        loss_train = loss_fn(t_p_train, target_train)

        t_p_val = model(data_val) # <1>
        loss_val = loss_fn(t_p_val, target_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            
linear_model = nn.Linear(11, 10)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-6)

# 학습 루프 실행
training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(), # <1>
    data_train = train_indices,
    data_val = val_indices, 
    target_train = target_train,
    target_val = target_val)

print(linear_model.weight)
print(linear_model.bias)


NameError: ignored

NameError: ignored

In [10]:
row = wine_data[0]

,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
0,7;0.27;0.36;20.7;0.045;45;170;1.001;3;0.45;8.8;6
1,6.3;0.3;0.34;1.6;0.049;14;132;0.994;3.3;0.49;9...
2,8.1;0.28;0.4;6.9;0.05;30;97;0.9951;3.26;0.44;1...
3,7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4...
4,7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4...
...,...
4893,6.2;0.21;0.29;1.6;0.039;24;92;0.99114;3.27;0.5...
4894,6.6;0.32;0.36;8;0.047;57;168;0.9949;3.15;0.46;...
4895,6.5;0.24;0.19;1.2;0.041;30;111;0.99254;2.99;0....
4896,5.5;0.29;0.3;1.1;0.022;20;110;0.98869;3.34;0.3...


In [9]:
col_list = next(csv.reader(open(wine_data), delimiter=';'))

TypeError: ignored

In [2]:
data = wineq[:, :-1]

In [ ]:
wine_data = pd.read_csv(wine_path, delimiter = ";")
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [4]:
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

IndexError: ignored

In [ ]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6.],
        [6.],
        [6.],
        ...,
        [6.],
        [7.],
        [6.]])

In [ ]:
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [ ]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [ ]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [ ]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [ ]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [ ]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

# col_list 추가
for i, args in enumerate(zip(bad_mean, mid_mean, good_mean)):
  print('{:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

                   0   7.60   6.89   6.73
                   1   0.33   0.28   0.27
                   2   0.34   0.34   0.33
                   3   6.39   6.71   5.26
                   4   0.05   0.05   0.04
                   5  53.33  35.42  34.55
                   6 170.60 141.83 125.25
                   7   0.99   0.99   0.99
                   8   3.19   3.18   3.22
                   9   0.47   0.49   0.50
                  10  10.34  10.26  11.42


In [ ]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()


(torch.Size([4898]), torch.bool, tensor(2727))

In [ ]:
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [ ]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

In [ ]:
# 종속변수/입력변수 나누기
x = wine_data.iloc[:,0:11]
y = wine_data['quality']

In [ ]:
# 라이브러리 불러오기
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
# 출력할때 warning 안뜨게하기

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('default')

In [ ]:
# 모델 학습
model = RandomForestClassifier(criterion='entropy',
                               n_estimators=3,oob_score=True)

model.fit(x,y)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
# 모델 예측 및 정확도 확인
y_pred = model.predict(x)
print(classification_report(y,y_pred))
print("정답률=",accuracy_score(y,y_pred))


              precision    recall  f1-score   support

           3       0.68      0.75      0.71        20
           4       0.78      0.91      0.84       163
           5       0.92      0.94      0.93      1457
           6       0.94      0.94      0.94      2198
           7       0.93      0.89      0.91       880
           8       0.95      0.82      0.88       175
           9       1.00      0.60      0.75         5

    accuracy                           0.92      4898
   macro avg       0.89      0.84      0.85      4898
weighted avg       0.92      0.92      0.92      4898

정답률= 0.9236423029808085


In [ ]:
# 모델안에 몇번째거를 그릴건지를 넣어야한다.

estimator = model.estimators_[0]

dot_data = export_graphviz(estimator, out_file=None,

                           feature_names=x.columns,

                           class_names=['q3','q4','q5','q6','q7','q8','q9'],

                           filled=True, rounded=True,

                           special_characters=True)

In [ ]:
# 그래프 그리기
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
from pydotplus import graph_from_dot_data
import os
from PIL import Image


graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.328639 to fit



TypeError: ignored

In [ ]:
# 모델학습
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion = "entropy", max_depth = 3)
model.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
# 모델 예측
y_pred = model.predict(x)
y_pred

array([5, 5, 6, ..., 6, 6, 6])

In [ ]:
# 정확도 확인
from sklearn.metrics import accuracy_score
accuracy_score(y,y_pred)

0.5195998366680278

In [ ]:
# 그래프 설정
from sklearn.tree import export_graphviz

dot_data = export_graphviz(model, out_file = None,
                           feature_names = x.columns,
                           class_names =
                           ['q3','q4','q5','q6','q7','q8','q9'],
                           filled = True, rounded = True,
                           special_characters = True)

In [ ]:
# 그래프 그리기
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
def plot_figure(index,column):
  plt.subplot(6,2,index)
  plt.title(column)
  plt.plot(wine_data[column])

  plt.figure(figsize=(10,10))


plt.tight_layout()

<Figure size 432x288 with 0 Axes>

In [ ]:
wine = wine_data[0:130] # 설명변수
wine_target = wine_data.target[0:130] # 목적변수

AttributeError: ignored